# Data Augmentation Using Generative Adversarial Networks (GANs)

Notebook to perform data augmentation using Generative Adversarial Networks (GANs). This notebook must be run after running the notebook `prepare_dataset_and_create_project_structure.ipynb` that should be available in the same directory.

### Prerequisites

1. Ensure that the Cityscapes dataset is downloaded and placed in the directory named **dataset**. More information about the dataset can be found in the notebook `prepare_dataset_and_create_project_structure.ipynb` that should be available in the same directory. 
2. Run all the cells in the notebook `prepare_dataset_and_create_project_structure.ipynb` to understand the dataset, process the dataset, create the project structure required to ensure correct results from the project.

### Check Python version

It is crucial to ensure that the notebook runs on the correct version of Python to guarantee proper functionality. 

In [1]:
import platform
assert (platform.python_version_tuple()[:2] >= ('3','7')), "[ERROR] The notebooks are tested on Python 3.7 and higher. Please updated your Python to evaluate the code"

### Check Notebook server has access to all required resources

In [2]:
from pathlib import Path

dataset_folder = Path("dataset")
dataset_folder = Path.joinpath(Path.cwd(), dataset_folder)

if not dataset_folder.exists():
    raise FileNotFoundError("[ERROR] Add `{}` folder in the current directory (`{}`)".format(dataset_folder.name, Path.cwd()))

In [3]:
dataset_preparation_notebook = Path("prepare_dataset_and_create_project_structure.ipynb")
dataset_preparation_notebook = Path.joinpath(Path.cwd(), dataset_preparation_notebook)

if not dataset_preparation_notebook.exists():
    raise FileNotFoundError("[ERROR] The notebook `{}` is unavailable in the current directory (`{}`). Please download and run the notebook `{}` before this notebook to ensure proper results.".format(dataset_preparation_notebook.name, Path.cwd(), dataset_preparation_notebook.name))

In [4]:
expected_datasets = ["test_dataset", "training_dataset", "validatation_dataset"]
expected_zipped_datasets_path = list()

for dataset in expected_datasets:
    dataset = Path.joinpath(dataset_folder, dataset)
    expected_zipped_datasets_path.append(dataset)
    if not dataset.exists():
        raise FileNotFoundError("[ERROR] The folder `{}` is unavailable. Please run the notebook `prepare_dataset_and_create_project_structure.ipynb` available in the current directory (`{}`) before running this notebook.".format(dataset.name, Path.cwd()))

### Introduction

One of the biggest bottlenecks in creating generalized deep learning models is a scarcity of high-quality data. The collection of high-quality data and its conversion is expensive. Most of the data collection methods are labor-intensive and error-prone, requiring considerable editing afterward to clean the data. Since large amounts of data are needed to achieve generalized deep learning models, standard data augmentation methods are routinely used to increase the dataset's generalizability. Data augmentation methods are also used when the datasets are imbalanced, improving the model's overall performance.

Generative Adversarial Networks, popularly known as GANs, are a novel method for data augmentation. The generation of artificial training data can not only be instrumental in situations such as imbalanced data sets, but it can also be useful when the original dataset contains sensitive information. In such cases, it is then desirable to avoid using the original data as much as possible (For example, Medical data).

This project proposes a GAN architecture based on this [paper](https://arxiv.org/abs/1611.07004) and evaluates the proposed GAN architecture's performance on a standard dataset. In this project, the Cityscape dataset is used to evaluate GAN architecture. The Cityscapes Dataset focuses on semantic understanding of urban street scenes. The dataset contains 5000 images with detailed annotations and 20000 images with coarse annotations apart from the original images. A sample image from the dataset is presented below:

![Sample Image from Cityscape Dataset](https://www.cityscapes-dataset.com/wordpress/wp-content/uploads/2015/07/stuttgart02-2040x500.png)

<center>Image Courtesy: Cityscapes Datatset (Link: https://www.cityscapes-dataset.com/)</center>

### Background Theory

With the advancements in deep learning, the most striking successes have involved discriminative models, usually those that map a high-dimensional, rich sensory input to a class label. These striking successes have primarily been based on the backpropagation and dropout algorithms, using piecewise linear units. However, deep generative models have had less impact due to the challenges of approximating many probabilistic computations that occur due to the usage of piecewise linear units in the generative context. 

Generative Adversarial Networks, popularly known as GANs, is a machine learning framework class that sidesteps these difficulties by pitting the generative model against an adversary. In other words, a GAN is a machine learning framework where two neural networks compete against each other in a zero-sum game (i.e., one network's gain is the other network's loss). The two networks in a GAN can be considered as a generator and a discriminator. The generator learns to create images that look real, while the discriminator learns to tell real images apart from fakes. Competition in this game drives both networks to improve their models until the counterfeits are indistinguishable from the genuine images. An overview of the Generative Adversarial Network is represented below:

![Overview of Generative Adversarial Network](https://developers.google.com/machine-learning/gan/images/gan_diagram.svg)

<center>Image Courtesy: Google Developers </center>